In [777]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [778]:
data = pd.read_csv('Data_Holland/mwl526_C.csv')
print(data.shape)
data.head()

(9027, 14)


,Pmin,Pmax,Pavg,Pref,Pcmin,Pcavg,Pcmax,Amin,Aavg,Amax,Ramin,Ramax,Ravg,Rain
0,-48,-47,-47.5,-47.50,-47.50,-47.50,-47.50,0.0,0.0,0.0,1.50,1.50,1.50,0
1,-47,-47,-47.0,-47.25,-47.25,-47.25,-47.25,0.0,0.0,0.0,1.25,1.25,1.25,0
2,-47,-47,-47.0,-47.00,-47.00,-47.00,-47.00,0.0,0.0,0.0,1.00,1.00,1.00,0
3,-47,-47,-47.0,-47.00,-47.00,-47.00,-47.00,0.0,0.0,0.0,1.00,1.00,1.00,0
4,-47,-47,-47.0,-47.00,-47.00,-47.00,-47.00,0.0,0.0,0.0,1.00,1.00,1.00,0


In [779]:
min1=np.array(data['Pmin'])
avg1=np.array(data['Pavg'])
max1=np.array(data['Pmax'])

min2=np.array(data['Amin'])
avg2=np.array(data['Aavg'])
max2=np.array(data['Amax'])

min3=np.array(data['Ramin'])
avg3=np.array(data['Ravg'])
max3=np.array(data['Ramax'])

unscaled_inputs_all = np.vstack((min3,avg3,max3))
unscaled_inputs_all = unscaled_inputs_all.T
targets_all =np.array(data['Rain'])
#targets_all.reshape(6434,1)
unscaled_inputs_all.shape

(9027, 3)

In [780]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_prior = np.delete(targets_all, indices_to_remove, axis = 0)

In [781]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
scaled_inputs.shape

(1868, 3)

In [782]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_prior[shuffled_indices]

In [783]:
#for unscaled data
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = unscaled_inputs_all[shuffled_indices]
shuffled_targets = targets_all[shuffled_indices]

In [784]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
tests_samples_count = int(samples_count-train_samples_count-validation_samples_count)

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
validation_targets = shuffled_targets[train_samples_count+validation_samples_count:]

tests_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
tests_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(tests_targets), tests_samples_count, np.sum(tests_targets)/tests_samples_count)

747 7221 0.10344827586206896
89 902 0.09866962305986696
89 904 0.09845132743362832


In [785]:
np.savez('RainH_data_train', inputs=train_inputs, targets=train_targets)
np.savez('RainH_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('RainH_data_test', inputs=tests_inputs, targets=tests_targets)

In [786]:
class Audiobooks_data_Reader():
    def  __init__(self, dataset, batch_size = None):  
        npz=np.load('RainH_data_{0}.npz'.format(dataset))
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    def  __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch+1) * self.batch_size)
        inputs_batch = self.inputs[batch_size]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        inputs_batch=inputs_batch.reshape(1,3)
        #targets_one_hot=targets_one_hot.reshape(2,100)
        return inputs_batch, targets_one_hot
    def  __iter__(self):
        return self

In [787]:
import tensorflow as tf

input_size = 3
output_size = 2

hidden_layer_size = 50

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32,[None, input_size])
targets = tf.placeholder(tf.float32,[None, output_size])

weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

outputs_1=tf.nn.softmax(tf.matmul(inputs,weights_1)+biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

outputs_2=tf.nn.softmax(tf.matmul(outputs_1,weights_2)+biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, hidden_layer_size])
biases_3 = tf.get_variable("biases_3", [hidden_layer_size])

outputs_3=tf.nn.softmax(tf.matmul(outputs_2,weights_3)+biases_3)

weights_4 = tf.get_variable("weights_4", [hidden_layer_size, output_size])
biases_4 = tf.get_variable("biases_4", [output_size])

outputs=tf.matmul(outputs_3,weights_4)+biases_4

loss=tf.nn.sigmoid_cross_entropy_with_logits(labels= targets,logits=outputs)

#loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels= targets)

mean_loss= tf.reduce_mean(loss)

optimize=tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

batch_size = 100

#batches_number = mnist.train._num_examples // batch_size

max_epochs = 100

prev_validation_loss = 9999999.

train_data = Audiobooks_data_Reader('train', batch_size)
validation_data = Audiobooks_data_Reader('validation')

for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss],
        feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
        feed_dict={inputs: input_batch, targets: target_batch})
    
    print('Epoch '+str(epoch_counter+1)+
        '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
        '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
        '. Validation accuracy : '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss=validation_loss
    
print('End of training.')

C:\Users\Evrad\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 1. Training loss: 0.458. Validation loss: 0.439. Validation accuracy : 90.15%
Epoch 2. Training loss: 0.430. Validation loss: 0.413. Validation accuracy : 90.15%
Epoch 3. Training loss: 0.407. Validation loss: 0.392. Validation accuracy : 90.15%
Epoch 4. Training loss: 0.389. Validation loss: 0.375. Validation accuracy : 90.15%
Epoch 5. Training loss: 0.375. Validation loss: 0.362. Validation accuracy : 90.15%
Epoch 6. Training loss: 0.364. Validation loss: 0.352. Validation accuracy : 90.15%
Epoch 7. Training loss: 0.355. Validation loss: 0.344. Validation accuracy : 90.15%
Epoch 8. Training loss: 0.349. Validation loss: 0.338. Validation accuracy : 90.15%
Epoch 9. Training loss: 0.344. Validation loss: 0.333. Validation accuracy : 90.15%
Epoch 10. Training loss: 0.340. Validation loss: 0.330. Validation accuracy : 90.15%
Epoch 11. Training loss: 0.338. Validation loss: 0.327. Validation accuracy : 90.15%
Epoch 12. Training loss: 0.336. Validation loss: 0.326. Validation accurac

In [788]:
test_data = Audiobooks_data_Reader('test')

for input_batch, target_batch in test_data:
    test_accuracy = sess.run([accuracy],
                            feed_dict={inputs: input_batch, targets: target_batch})
test_accuracy_percent = test_accuracy[0] * 100.
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 90.15%


In [789]:
test_data

In [792]:
outputs.dtype

tf.float32